In [0]:
#creating and implementing CNNs
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [0]:
class Net(nn.Module):
  def __init__(self):
      super(Net, self).__init__()

      #creating the network

      self.conv1=nn.Conv2d(1, 6, 3)
      self.conv2=nn.Conv2d(6, 16, 3)

      self.fc1=nn.Linear(16*6*6, 120)
      self.fc2=nn.Linear(120, 84)
      self.fc3=nn.Linear(84, 10)

  def num_flat_features(self, x):
      size=x.size()[1:]
      num_features=1
      for s in size:
        num_features*=s

      return num_features
    #defining the forward prop

  def forward(self, x):
      x=F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
      x=F.max_pool2d(F.relu(self.conv2(x)), 2)
      x=x.view(-1, self.num_flat_features(x))
      x=F.relu(self.fc1(x))
      x=F.relu(self.fc2(x))
      x=self.fc3(x)
      return x

 

    

In [3]:
net=Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
params=list(net.parameters())
print('no. of parameters: ', len(params))
print('parameters: ', params[0].size())

no. of parameters:  10
parameters:  torch.Size([6, 1, 3, 3])


In [0]:
#trying the CONV net 

input=torch.randn(1, 1, 32, 32)     #one 32x32 size greyscale kind of image
out=net(input)
print(out)

tensor([[ 0.0099,  0.0822, -0.0683, -0.0404,  0.0244, -0.0832, -0.0691,  0.1680,
          0.0638, -0.0077]], grad_fn=<AddmmBackward>)


In [0]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [0]:
#calculating loss
#MSE:Mean Squared Error
output=net(input)
target=torch.randn(10)    #dummy target 
target=target.view(1, -1)
criterion=nn.MSELoss()
loss=criterion(output, target)
print(loss)

tensor(1.2259, grad_fn=<MseLossBackward>)


In [0]:
#getting a few grad fxns.
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [0]:
#getting the grads and backprop

print('conv1.bias.grad before backward and zero_grad: ')
print(net.conv1.bias.grad)

net.zero_grad()

print('conv1.bias.grad before backward and after zero_grad: ')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward and zero_grad: ')
print(net.conv1.bias.grad)

conv1.bias.grad before backward and zero_grad: 
tensor([ 0.0241,  0.0208,  0.0440, -0.0041, -0.0034, -0.0344])
conv1.bias.grad before backward and after zero_grad: 
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward and zero_grad: 
tensor([-0.0094, -0.0029, -0.0070, -0.0030, -0.0035,  0.0008])


In [0]:
#updating the parameters3

optimiser=optim.SGD(net.parameters(), lr=0.01)    #optimiser created

optimiser.zero_grad()
output=net(input)
loss=criterion(output, target)
loss.backward()
optimiser.step()      #this does the update